# Análisis RFM (Recencia, Frecuencia, Monetario)

Este notebook replica el análisis RFM implementado en `src/mi_analisis.py`.

Objetivos:\n- Calcular RFM por cliente\n- Generar una tabla resumen y dos visualizaciones: conteo por segmento y top productos.

In [1]:
# Imports y configuración
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
sns.set(style="whitegrid")

ROOT = Path("..").resolve()
REPORT_DIR = ROOT / 'reports'
REPORT_DIR.mkdir(exist_ok=True)

In [2]:
# Funciones auxiliares (adaptadas del script)
from typing import Optional

def locate_file(names, search_paths):
    for base in search_paths:
        if not base.exists():
            continue
        for name in names:
            p = base / name
            if p.exists():
                return p
        for p in base.rglob('*'):
            if p.name in names:
                return p
    return None

def load_df(p: Path) -> pd.DataFrame:
    encodings = ['utf-8', 'latin1', 'cp1252']
    seps = [',', ';', '\t']
    last_err = None
    for enc in encodings:
        for sep in seps:
            try:
                return pd.read_csv(p, encoding=enc, sep=sep, engine='python')
            except Exception as e:
                last_err = e
    try:
        return pd.read_csv(p, engine='python')
    except Exception:
        raise last_err if last_err is not None else RuntimeError('No se pudo leer')

In [3]:
# Heurísticos para detectar columnas relevantes
def find_date_column(df: pd.DataFrame) -> Optional[str]:
    candidates = ['fecha','fecha_venta','fechaVenta','date','created_at','fecha_hora']
    cols = [c.lower() for c in df.columns]
    for cand in candidates:
        if cand.lower() in cols:
            return df.columns[cols.index(cand.lower())]
    for c in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[c]):
            return c
    for c in df.columns:
        if 'fecha' in c.lower() or 'date' in c.lower():
            return c
    return None

def find_customer_col(df: pd.DataFrame) -> Optional[str]:
    candidates = ['cliente_id','id_cliente','idcliente','cliente','customer_id']
    cols = [c.lower() for c in df.columns]
    for cand in candidates:
        if cand.lower() in cols:
            return df.columns[cols.index(cand.lower())]
    return None

def find_total_col(df: pd.DataFrame) -> Optional[str]:
    candidates = ['total','importe','monto','total_venta','valor']
    cols = [c.lower() for c in df.columns]
    for cand in candidates:
        if cand.lower() in cols:
            return df.columns[cols.index(cand.lower())]
    return None

In [4]:
# Implementación RFM (agregación)
def compute_rfm(ventas: pd.DataFrame) -> pd.DataFrame:
    date_col = find_date_column(ventas)
    cust_col = find_customer_col(ventas)
    total_col = find_total_col(ventas)
    if date_col is None or cust_col is None:
        raise RuntimeError('No se encontraron columnas de fecha o cliente en el DataFrame de ventas')
    ventas = ventas.copy()
    ventas[date_col] = pd.to_datetime(ventas[date_col], errors='coerce')
    ventas = ventas.dropna(subset=[date_col, cust_col])
    if total_col is not None and total_col in ventas.columns:
        ventas['_total'] = pd.to_numeric(ventas[total_col], errors='coerce').fillna(0.0)
    else:
        ventas['_total'] = 0.0
    ventas['_customer'] = ventas[cust_col].astype(str)
    ventas['_date'] = ventas[date_col]
    reference_date = ventas['_date'].max() + pd.Timedelta(days=1)
    agg = ventas.groupby('_customer').agg(
        recency=('_date', lambda x: (reference_date - x.max()).days),
        frequency=('_date', 'count'),
        monetary=('_total', 'sum')
    ).reset_index()
    # scores por cuartiles
    agg['r_score'] = pd.qcut(agg['recency'].rank(method='first'), 4, labels=[4,3,2,1]).astype(int)
    agg['f_score'] = pd.qcut(agg['frequency'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
    agg['m_score'] = pd.qcut(agg['monetary'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
    agg['rfm_score'] = agg['r_score'].astype(str) + agg['f_score'].astype(str) + agg['m_score'].astype(str)
    def label_row(row):
        s = (row['r_score'] + row['f_score'] + row['m_score']) / 3
        if s >= 3.5:
            return 'Champions'
        if s >= 2.5:
            return 'Loyal'
        if s >= 1.5:
            return 'Needs Attention'
        return 'At Risk'
    agg['segment'] = agg.apply(label_row, axis=1)
    return agg.sort_values(['monetary'], ascending=False)

In [5]:
# Ejecución: localizar archivos y correr RFM (si existen)
SEARCH_PATHS = [
    ROOT / 'entrega2' / 'data' / 'csv' / 'origin',
    ROOT / 'entrega2' / 'data' / 'csv',
    ROOT / 'db',
    ROOT,
]
filenames = {
    'ventas': ['ventas.csv','venta.csv'],
    'detalle': ['detalle_ventas.csv','detalle.csv'],
    'clientes': ['clientes.csv','cliente.csv'],
    'productos': ['productos.csv','producto.csv'],
}
ventas_p = locate_file(filenames['ventas'], SEARCH_PATHS)
if ventas_p is None:
    print('No se encontraron archivos de ventas en las rutas buscadas. Coloca los CSV en db/ o entrega2/data/csv/origin y vuelve a intentar.')
else:
    ventas = load_df(ventas_p)
    try:
        agg = compute_rfm(ventas)
    except Exception as e:
        print('Error calculando RFM:', e)
    else:
        out_csv = REPORT_DIR / 'rfm_summary.csv'
        agg.to_csv(out_csv, index=False)
        print('Guardado RFM resumen en:', out_csv)

Guardado RFM resumen en: /home/jiss/Documentos/repositories/guayerd-e1/reports/rfm_summary.csv
